<a href="https://colab.research.google.com/github/XoomyBot/XoomyBot/blob/master/functie_shapesConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EAVISE/Beeldinterpretatie_Lab_Template/blob/master/colab_template.ipynb)

In [1]:
# These lines will update the dependencies in this Jupyter notebook
!pip install -qq -U 'datascience' 'albumentations==0.3.3'
!pip install -qq  'opencv-python==4.1.1.26' --force-reinstall
!pip install -qq 'ipywebrtc' 'mpld3'

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import *
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import copy
import math

import mpld3
from mpld3 import plugins
import time
from tqdm import tqdm_notebook as tqdm

font = cv2.FONT_HERSHEY_COMPLEX

     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 92kB 5.2MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 634kB 15.3MB/s 
     |████████████████████████████████| 28.7MB 108kB/s 
     |████████████████████████████████| 20.2MB 102kB/s 
     |████████████████████████████████| 1.1MB 9.9MB/s 
     |████████████████████████████████| 798kB 54.5MB/s 


In [0]:
def Moves(image):
  matrix = 10
  kernel = np.ones((matrix,matrix),np.uint8)
  imagePath = image
  gray = cv2.imread(imagePath, cv2.IMREAD_GRAYSCALE)
  #plt.subplot(231),plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB)),plt.title('gray')
  #plt.axis("off")
  ret, th = cv2.threshold(gray, 150, 255, 0)
  #plt.subplot(232),plt.imshow(cv2.cvtColor(th, cv2.COLOR_BGR2RGB)),plt.title('Threshold')
  #plt.axis("off")
  opened = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
  #plt.subplot(233),plt.imshow(cv2.cvtColor(opened, cv2.COLOR_BGR2RGB)),plt.title('opened')
  #plt.axis("off")
  closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
  #plt.subplot(234),plt.imshow(cv2.cvtColor(closed, cv2.COLOR_BGR2RGB)),plt.title('closed')
  #plt.axis("off")
  image = gray
  thresh = closed
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  points = []
  for cnt in contours:
      approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
      points.append(approx)
      cv2.drawContours(image, [approx], 0, (0), 5) 
  #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)),plt.title('+contours')
  #plt.axis("off")
  cont = []
  for u in points: 
    shape = []
    x = 0
    for v in u:
      for w in v:
        if x == 0:
          eerste_x = w[0] #het onthouden van de start om het contour te sluiten(anders zou er 1lijn vergeten worden)
          eerste_y = w[1]
          eerste = w
        shape.append(w)
      x = 1
    shape.append(eerste)
    cont.append(shape)
  scaler = 100
  x0 = 0 #oorsprong
  y0 = 0
  theta0 = 0 #rechts
  moves = []
  count = 0
  for a in cont:
    for b in a:
      x1 = b[0]
      y1 = b[1]
      Dy = y1-y0
      Dx = x1-x0
      theta = math.atan(Dy/Dx)*(360/(2*math.pi))#omvormen naar graden
      if count < 2:
        dtheta = theta
      else:
        dtheta = theta-theta0 #foutje met nan!!
      if Dx < 0:
        if dtheta > 0:
          dtheta = dtheta-180 #graden!!
          theta = theta-180
        else:
          dtheta = dtheta+180 #graden!!
          theta = theta+180
      distance = ((Dx**2 + Dy**2)**0.5)/scaler
      hoek = {
          "func": "turn",
          "distance": 0,
          "angle": dtheta
      }
      vooruit =	{
          "func": "straight",
          "distance": distance,
          "angle": 0
      }
      moves.append(hoek)
      moves.append(vooruit)
      x0 = x1
      y0 = y1
      theta0 = theta
      dtheta0 = dtheta
      count = count+1
      #print (hoek)
      #print (vooruit)
      #print("thetaEff= " + str(theta0) + "\nthetaRel= " + str(dtheta) + "\ndistance= " + str(distance) + "\n-------------------------------")
    #print("-----------------------------------------------------")
  #print(moves)
  return moves

In [7]:
moves = Moves("/content/sample_data/vierkant.jpg")
print(moves)

[{'func': 'turn', 'distance': 0, 'angle': nan}, {'func': 'straight', 'distance': 0.0, 'angle': 0}, {'func': 'turn', 'distance': 0, 'angle': 90.0}, {'func': 'straight', 'distance': 3.99, 'angle': 0}, {'func': 'turn', 'distance': 0, 'angle': -90.0}, {'func': 'straight', 'distance': 3.99, 'angle': 0}, {'func': 'turn', 'distance': 0, 'angle': -90.0}, {'func': 'straight', 'distance': 3.99, 'angle': 0}, {'func': 'turn', 'distance': 0, 'angle': -90.0}, {'func': 'straight', 'distance': 3.99, 'angle': 0}]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in int_scalars
